#**LSTM for Text Classification**

##**Importing Required Libraries**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import pad_sequences
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

##**Reading dataset and doing Pre-processing**

In [3]:
data = pd.read_csv('spam.csv',delimiter=',',encoding='latin-1')
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [4]:
data.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True)

In [5]:
data.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
X = data.v2
Y = data.v1
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [7]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y)

In [8]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = pad_sequences(sequences,maxlen=max_len)

##**Creating Model & Adding Layers (LSTM, Dense-(Hidden Layers), Output)**

In [13]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [14]:
model = RNN()
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 150)]             0         
                                                                 
 embedding_1 (Embedding)     (None, 150, 50)           50000     
                                                                 
 lstm_1 (LSTM)               (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation_2 (Activation)   (None, 256)               0         
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257 

##**Compiling the Model**

In [15]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

##**Fitting the Model**

In [16]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,validation_split=0.2)

Epoch 1/10
27/27 [==============================] - 12s 315ms/step - loss: 0.4571 - accuracy: 0.8570 - val_loss: 0.3004 - val_accuracy: 0.8660
Epoch 2/10
27/27 [==============================] - 8s 284ms/step - loss: 0.1825 - accuracy: 0.9303 - val_loss: 0.1139 - val_accuracy: 0.9725
Epoch 3/10
27/27 [==============================] - 8s 284ms/step - loss: 0.0543 - accuracy: 0.9844 - val_loss: 0.0847 - val_accuracy: 0.9785
Epoch 4/10
27/27 [==============================] - 7s 275ms/step - loss: 0.0350 - accuracy: 0.9889 - val_loss: 0.0663 - val_accuracy: 0.9785
Epoch 5/10
27/27 [==============================] - 7s 275ms/step - loss: 0.0205 - accuracy: 0.9946 - val_loss: 0.0647 - val_accuracy: 0.9773
Epoch 6/10
27/27 [==============================] - 8s 279ms/step - loss: 0.0122 - accuracy: 0.9976 - val_loss: 0.0739 - val_accuracy: 0.9809
Epoch 7/10
27/27 [==============================] - 7s 277ms/step - loss: 0.0094 - accuracy: 0.9982 - val_loss: 0.0736 - val_accuracy: 0.9785
Epoch

##**Saving the Model**

In [20]:
model.save('spam.h5')

##**Testing the Model**

In [22]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=max_len)

In [23]:
accuracy = model.evaluate(test_sequences_matrix,Y_test)

44/44 [==============================] - 2s 54ms/step - loss: 0.0851 - accuracy: 0.9828


In [24]:
print('Test set\n  Loss: {:0.4f}\n  Accuracy: {:0.4f}'.format(accuracy[0],accuracy[1]))

Test set
  Loss: 0.0851
  Accuracy: 0.9828
